In [20]:
import pandas as pd
import os

In [43]:
df = pd.DataFrame()

for result in os.listdir("./result"):
    result = 'result/' + result
    temp = pd.read_csv(result)
    df = pd.concat([df, temp])

In [44]:
df.loc[df['Start'] == '2018-03-01'].sort_values('SQN', ascending=False).head(10)

,Pair,Timeframe,Start,End,Strategy,Period,Final value,%,Total win,Total loss,SQN
0,BTCUSDT,1w,2018-03-01,2020-11-15,RSI,10,11519.086,15.191,2,0,3.08
1,BTCUSDT,1w,2018-03-01,2020-11-15,RSI,11,11637.561,16.376,2,0,2.84
7,ETHUSDT,4h,2018-03-01,2020-11-15,SMA,17,12711.644,27.116,95,250,2.42
5,ETHUSDT,4h,2018-03-01,2020-11-15,SMA,15,12618.773,26.188,100,270,2.38
1,ETHUSDT,6h,2018-03-01,2020-11-15,SMA,11,12579.214,25.792,91,207,2.35
8,ETHUSDT,4h,2018-03-01,2020-11-15,SMA,18,12569.414,25.694,86,254,2.22
2,ETHUSDT,6h,2018-03-01,2020-11-15,SMA,12,12387.524,23.875,87,209,2.21
4,ETHUSDT,4h,2018-03-01,2020-11-15,SMA,14,12407.632,24.076,103,300,2.20
6,ETHUSDT,4h,2018-03-01,2020-11-15,SMA,16,12434.159,24.342,98,266,2.19
0,ETHUSDT,6h,2018-03-01,2020-11-15,SMA,10,12397.067,23.971,97,224,2.18


In [45]:
df.sort_values('SQN', ascending=False).head(10)

,Pair,Timeframe,Start,End,Strategy,Period,Final value,%,Total win,Total loss,SQN
0,BTCUSDT,1w,2017-01-01,2020-12-31,RSI,10,11519.086,15.191,2,0,3.08
0,BTCUSDT,1w,2018-03-01,2020-11-15,RSI,10,11519.086,15.191,2,0,3.08
1,ETHUSDT,6h,2017-01-01,2020-12-31,SMA,11,14874.364,48.824,122,243,3.05
2,BTCUSDT,8h,2017-01-01,2020-12-31,SMA,12,13813.646,38.188,83,178,2.89
2,ETHUSDT,6h,2017-01-01,2020-12-31,SMA,12,14564.124,45.720,113,249,2.88
1,BTCUSDT,12h,2017-01-01,2020-12-31,SMA,11,13937.049,39.309,56,112,2.87
1,BTCUSDT,1w,2018-03-01,2020-11-15,RSI,11,11637.561,16.376,2,0,2.84
1,BTCUSDT,1w,2017-01-01,2020-12-31,RSI,11,11637.561,16.376,2,0,2.84
3,BTCUSDT,6h,2017-01-01,2020-12-31,SMA,13,13745.854,37.239,96,240,2.82
0,ETHUSDT,6h,2017-01-01,2020-12-31,SMA,10,14330.644,43.384,129,265,2.82


In [46]:
df.sort_values('%', ascending=False).head(10)

,Pair,Timeframe,Start,End,Strategy,Period,Final value,%,Total win,Total loss,SQN
1,ETHUSDT,6h,2017-01-01,2020-12-31,SMA,11,14874.364,48.824,122,243,3.05
3,BTCUSDT,12h,2017-01-01,2020-12-31,SMA,13,14773.328,47.351,47,108,2.25
19,BTCUSDT,6h,2017-01-01,2020-12-31,SMA,29,14672.107,46.487,48,140,2.28
2,ETHUSDT,6h,2017-01-01,2020-12-31,SMA,12,14564.124,45.720,113,249,2.88
0,ETHUSDT,8h,2017-01-01,2020-12-31,SMA,10,14565.665,45.648,95,198,2.62
0,ETHUSDT,3d,2017-01-01,2020-12-31,SMA,10,15467.598,44.964,12,14,1.80
20,BTCUSDT,6h,2017-01-01,2020-12-31,SMA,30,14517.592,44.944,48,144,2.21
19,ETHUSDT,1d,2017-01-01,2020-12-31,SMA,29,14491.179,44.625,14,34,1.71
18,BTCUSDT,6h,2017-01-01,2020-12-31,SMA,28,14471.304,44.482,50,149,2.18
5,BTCUSDT,12h,2017-01-01,2020-12-31,SMA,15,14812.219,44.266,42,95,2.27
